In [4]:
from tensorflow.python.summary.summary_iterator import summary_iterator
import matplotlib.pylab as plt
import glob
import os
import numpy as np
import pandas as pd 
from scipy import stats
import json

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/s

In [64]:
def test_auc(tf_path):
    """Take a tf output path and create a list of values for the validtion auc of an experement"""
    auc = []
    for event in summary_iterator(tf_path):
        for value in event.summary.value:
            if value.tag == "final/test_auc":
                auc.append(value.simple_value)
    return auc

def get_auc_scores_from_file(root_dir):
    try:
        if isinstance(root_dir, str):
            all_aucs = []
            file_extension = "*.gpu1"

            joined_dir = "/mnt/ncshare/ozkilim/BRCA/results_HGSOC_multimodal/" + root_dir
            # Recursively traverse the directory and its subdirectories
            for subdir, _, files in os.walk(joined_dir):
                # Use glob to find all files with the specified extension
                for file in glob.glob(os.path.join(subdir, file_extension)):
                    auc = test_auc(file)
                    all_aucs.append(auc[0])
        else:
            all_aucs = root_dir.copy()
        
    except:
        all_aucs =  np.array([0,0,0,0,0,0,0,0,0])

    
    return all_aucs

def get_eval_results(eval_path):

    joined_dir = "/mnt/ncshare/ozkilim/BRCA/eval_results/" + eval_path
    try:
        all_aucs = pd.read_csv(joined_dir)
        all_aucs = all_aucs["test_auc"]
    except:
        return np.array([0,0,0,0,0,0,0,0,0])
    
    return all_aucs


def process_auc_files(results_dict):
    # Initialize an empty DataFrame to store all AUC scores
    # Loop through each file and extract AUC scores
    all_auc_scores = []
    # loop overdict...  

    items_list = list(results_dict)
    # Loop over the dictionary by index
    for i in range(len(items_list)):
                
        auc_scores_AUB = get_auc_scores_from_file(list(results_dict[i].values())[0][0]) #always spulls same key
        auc_scores_Mayo = get_auc_scores_from_file(list(results_dict[i].values())[0][1])
        category = list(results_dict[i].values())[0][2]
        embedder = list(results_dict[i].values())[0][3]

        # get stats here... 
        auc_AUB = round(np.mean(auc_scores_AUB),3)
        std_AUB = round(np.std(auc_scores_AUB),3)

        auc_Mayo = round(np.mean(auc_scores_Mayo),3)
        std_Mayo = round(np.std(auc_scores_Mayo),3)

        row = {"model":list(results_dict[i].keys())[0], "UAB":auc_AUB.astype(str)+"±"+std_AUB.astype(str), "Mayo":auc_Mayo.astype(str)+"±"+std_Mayo.astype(str),"category":category,"embedder":embedder}

        all_auc_scores.append(row)
    
    all_auc_scores = pd.DataFrame(all_auc_scores)


    # Sort the DataFrame based on 'Mean AUC' in descending order
    sorted_auc_summary = all_auc_scores.sort_values(by='Mayo', ascending=True)
    
    return sorted_auc_summary



In [102]:
tissue_type = "metastatic"


#load dynamically.

rf_primary_UAB = np.array([0.703125,   0.78645833, 0.75  ,  0.64583333, 0.66145833])
rf_primary_FHCRC = np.array([0.73090909, 0.67676768, 0.61868687, 0.47727273, 0.55681818])
rf_primary_Mayo = np.array([0.71428571, 0.5974026,  0.58443058 ,0.5974026 , 0.64285714])


rf_metastatic_UAB = np.array([0.73529412, 0.80252101, 0.72689076 ,0.7605042 , 0.79831933])
rf_metastatic_FHCRC = np.array([0.69537830, 0.79201681, 0.73319328 ,0.75210084, 0.7710084 ])
rf_metastatic_Mayo = np.array([0.71428571, 0.78968254, 0.71031746, 0.83730309, 0.8234127 ])



# Load the dictionary from the JSON file
with open('classical_omics_results_metastatic.json', 'r') as json_file:
    metastatic_omics = json.load(json_file)

with open('classical_omics_results_primary.json', 'r') as json_file:
    primary_omics = json.load(json_file)


#import results data for all runs.
results_dict = [
                # "Random forest":[rf_primary_UAB,rf_primary_FHCRC, rf_primary_Mayo,"Omics"],
                {"Ensemble \cite{chowdhury2023proteogenomic}":[metastatic_omics["UAB_aucs_metastatic"],metastatic_omics["Mayo_aucs_metastatic"],"Omics","CTransPath"]},

                {"clam\_sb \cite{lu2021data}":["HGSOC_clam_sb_ViT_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_clam_sb_ViT_Mayo_"+tissue_type+"_15_epocs_s1","WSI","Lunit-Dino"]},
                {"clam\_sb \cite{lu2021data}" :["HGSOC_clam_sb_OV_ViT_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_clam_sb_OV_ViT_Mayo_"+tissue_type+"_15_epocs_s1","WSI","OV-Dino"]},
                {"clam\_sb \cite{lu2021data}":["HGSOC_clam_sb_CTransPath_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_clam_sb_CTransPath_Mayo_"+tissue_type+"_15_epocs_s1","WSI", "CTransPath"]},
                
                
                {"SurvPath \cite{jaume2023modeling}":["HGSOC_SurvPath_ViT_UAB_"+tissue_type+"_15_epocs_s1", "HGSOC_SurvPath_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","Lunit-Dino"]},
                {"SurvPath \cite{jaume2023modeling}":["HGSOC_SurvPath_OV_ViT_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_SurvPath_OV_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","OV-Dino"]},
                {"SurvPath \cite{jaume2023modeling}":["HGSOC_SurvPath_CTransPath_UAB_"+tissue_type+"_15_epocs_s1", "HGSOC_SurvPath_CTransPath_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","CTransPath"]},
                

                {"MCAT PPI \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_ViT_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_MCAT_Surv_ViT_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal", "Lunit-Dino"]},
                {"MCAT PPI \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_OV_ViT_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_MCAT_Surv_OV_ViT_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal","OV-Dino"]},
                {"MCAT PPI \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_CTransPath_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_MCAT_Surv_CTransPath_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal","CTransPath"]},


                {"MCAT \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_ViT_UAB_"+tissue_type+"_15_epocs_s1", "HGSOC_MCAT_Surv_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","Lunit-Dino"]},
                {"MCAT \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_OV_ViT_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_MCAT_Surv_OV_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","OV-Dino"]},
                {"MCAT \cite{chen2021multimodal}":["HGSOC_MCAT_Surv_CTransPath_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_MCAT_Surv_CTransPath_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","CTransPath"]},


                {"PorpoiseMMF \cite{chen2022pan}":["HGSOC_PorpoiseMMF_ViT_UAB_"+tissue_type+"_15_epocs_s1", "HGSOC_PorpoiseMMF_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","Lunit-Dino"]},
                {"PorpoiseMMF \cite{chen2022pan}":["HGSOC_PorpoiseMMF_OV_ViT_UAB_"+tissue_type+"_15_epocs_s1", "HGSOC_PorpoiseMMF_OV_ViT_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","OV-Dino"]},
                {"PorpoiseMMF \cite{chen2022pan}":["HGSOC_PorpoiseMMF_CTransPath_UAB_"+tissue_type+"_15_epocs_s1","HGSOC_PorpoiseMMF_CTransPath_Mayo_"+tissue_type+"_15_epocs_s1","Multimodal","CTransPath"]},

                {"SurvPath 60 PPI \cite{jaume2023modeling}":["HGSOC_SurvPath_ViT_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_SurvPath_ViT_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal","Lunit-Dino"]},
                {"SurvPath 60 PPI \cite{jaume2023modeling}":["HGSOC_SurvPath_OV_ViT_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_SurvPath_OV_ViT_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal","OV-Dino"]},
                {"SurvPath 60 PPI \cite{jaume2023modeling}":["HGSOC_SurvPath_CTransPath_UAB_"+tissue_type+"_15_epocs_PPI_60_s1","HGSOC_SurvPath_CTransPath_Mayo_"+tissue_type+"_15_epocs_PPI_60_s1","Multimodal","CTransPath"]},
                
                ]

df = process_auc_files(results_dict)
df.head(30)

,model,UAB,Mayo,category,embedder
11,MCAT \cite{chen2021multimodal},0.586±0.038,0.626±0.026,Multimodal,OV-Dino
2,clam\_sb \cite{lu2021data},0.65±0.048,0.639±0.026,WSI,OV-Dino
9,MCAT PPI \cite{chen2021multimodal},0.65±0.021,0.654±0.05,Multimodal,CTransPath
7,MCAT PPI \cite{chen2021multimodal},0.534±0.027,0.691±0.031,Multimodal,Lunit-Dino
10,MCAT \cite{chen2021multimodal},0.641±0.024,0.701±0.022,Multimodal,Lunit-Dino
17,SurvPath 60 PPI \cite{jaume2023modeling},0.736±0.029,0.706±0.024,Multimodal,OV-Dino
8,MCAT PPI \cite{chen2021multimodal},0.55±0.035,0.712±0.031,Multimodal,OV-Dino
18,SurvPath 60 PPI \cite{jaume2023modeling},0.762±0.047,0.714±0.021,Multimodal,CTransPath
15,PorpoiseMMF \cite{chen2022pan},0.731±0.023,0.715±0.046,Multimodal,CTransPath
3,clam\_sb \cite{lu2021data},0.543±0.062,0.738±0.038,WSI,CTransPath


In [105]:

# Pivot the DataFrame
df_pivot = df.pivot_table(index=['model', 'category'], columns='embedder', values=['Mayo', 'UAB'], aggfunc='first').reset_index()
df_pivot.columns = [' '.join(col).strip() for col in df_pivot.columns.values]

def format_highest_values(df):
    for col in df.columns[2:]:
        highest_value = df[col].max()
        df[col] = df[col].apply(lambda x: f'\\textbf{{{x}}}' if x == highest_value else x)
    return df


# Function to generate LaTeX table
def generate_latex_table(df_pivot):
    # Start the table and add the header
    latex_str = "\\begin{table}[ht]\n\\centering\n\\begin{tabular}{cc|ccc|ccc}\n\\toprule\n"
    latex_str += " & \\multicolumn{1}{c}{Model} & \\multicolumn{3}{c}{Mayo} & \\multicolumn{3}{c}{UAB} \\\\\n"
    latex_str += "\\midrule\n"
    latex_str += " &  & CTransPath \cite{wang2022transformer} & Lunit-Dino \cite{kang2023benchmarking} & OV-Dino (ours) & CTransPath & Lunit-Dino & OV-Dino \\\\\n"
    latex_str += "\\midrule\n"

    # Add rows from the DataFrame
    for category, group_df in df_pivot.groupby('category'):
        group_len = len(group_df)
        latex_str += f"\\multirow{{{group_len}}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{category}}}}} \n"
        for _, row in group_df.iterrows():
            model = row['model']
            values = ' & '.join(str(x) for x in row[2:])
            latex_str += f" & {model} & {values} \\\\\n"
        latex_str += "\\midrule\n"

    latex_str += "\\bottomrule\n\\end{tabular}\n\\caption{Your caption here}\n\\end{table}"

    return latex_str

# Generate LaTeX table
latex_table = generate_latex_table(df_pivot)
print(latex_table)

\begin{table}[ht]
\centering
\begin{tabular}{cc|ccc|ccc}
\toprule
 & \multicolumn{1}{c}{Model} & \multicolumn{3}{c}{Mayo} & \multicolumn{3}{c}{UAB} \\
\midrule
 &  & CTransPath \cite{wang2022transformer} & Lunit-Dino \cite{kang2023benchmarking} & OV-Dino (ours) & CTransPath & Lunit-Dino & OV-Dino \\
\midrule
\multirow{5}{*}{\rotatebox[origin=c]{90}{Multimodal}} 
 & MCAT PPI \cite{chen2021multimodal} & 0.654±0.05 & 0.691±0.031 & 0.712±0.031 & 0.65±0.021 & 0.534±0.027 & 0.55±0.035 \\
 & MCAT \cite{chen2021multimodal} & 0.741±0.03 & 0.701±0.022 & 0.626±0.026 & 0.69±0.014 & 0.641±0.024 & 0.586±0.038 \\
 & PorpoiseMMF \cite{chen2022pan} & 0.715±0.046 & 0.76±0.026 & 0.889±0.034 & 0.731±0.023 & 0.716±0.032 & 0.705±0.042 \\
 & SurvPath 60 PPI \cite{jaume2023modeling} & 0.714±0.021 & 0.763±0.02 & 0.706±0.024 & 0.762±0.047 & 0.731±0.047 & 0.736±0.029 \\
 & SurvPath \cite{jaume2023modeling} & 0.789±0.02 & 0.851±0.026 & 0.785±0.053 & 0.705±0.04 & 0.682±0.024 & 0.634±0.046 \\
\midrule
\multirow{1}{